# Attribution CIs for Integrated Gradients

(Average absolute) integrated gradients, designed specifically for the linear setting

In [1]:
import numpy as np
import torch
from sklearn.linear_model import LinearRegression
from uncertainty.bootstrapCoefficients import bootstrapCis
from uncertainty.analyticLinearRegressionCIs import analyticLinearCis
from uncertainty.DataGeneration import default_data, linearRegression_normal
from uncertainty.glrtTorch import glrtTorchCis, MSE
from uncertainty.torch_linear import TorchLinear
from uncertainty.glrt_stat import bootstrapGLRTcis
from uncertainty.IG_linearModels import integratedGradients_Linear, bootstrapIntegratedGradients_Linear, integratedGradients_LinearAnalytical
from attributionpriors.pytorch_ops import ExpectedGradientsModel

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
X, y = default_data()

## Test analytical and bootstrap IG CIs

In [4]:
# Sklearn Linear Regression
LR = LinearRegression(fit_intercept=False)
LR.fit(X, y)
print("Coefficients:", LR.coef_)
print("Integrated gradients:", integratedGradients_Linear(LR.coef_, X))

print("Bootstrapping")
lcb_LR, ucb_LR = bootstrapIntegratedGradients_Linear(LinearRegression, X, y, alpha=0.05, replicates=1000)
print("Lower bounds:", lcb_LR, "\nUpper bounds:", ucb_LR)

print("Analytical")
lcb_LR_a, ucb_LR_a = integratedGradients_LinearAnalytical(LR, X, y, alpha=0.05)
print("Lower bounds:", lcb_LR_a, "\nUpper bounds:", ucb_LR_a)

Coefficients: [0.89608084 1.07997608]
Integrated gradients: [0.80678129 0.88687844]
Bootstrapping
Lower bounds: [0.64682205 0.7488702 ] 
Upper bounds: [0.93818827 1.0281889 ]
Analytical
Lower bounds: [0.66930992 0.75537646] 
Upper bounds: [0.94425266 1.01838043]


In [5]:
# Torch Linear Regression
TL = TorchLinear(lr=0.3,max_iter=30,fit_intercept=False)
TL.fit(X,y)

print("Coefficients:", TL.coef_)
print("Integrated gradients:", integratedGradients_Linear(TL.coef_, X))

# Takes ~4min
print("Bootstrapping")
lcb_TL, ucb_TL = bootstrapIntegratedGradients_Linear(lambda:TorchLinear(lr=0.3,max_iter=30), X=X, y=y, alpha=0.05, replicates=1000)
print("Lower bounds:", lcb_TL, "\nUpper bounds:", ucb_TL)

print("Analytical")
lcb_TL_a, ucb_TL_a = integratedGradients_LinearAnalytical(TL, X, y, alpha=0.05)
print("Lower bounds:", lcb_TL_a, "\nUpper bounds:", ucb_TL_a)

Coefficients: [0.8960828 1.0799739]
Integrated gradients: [0.80678307 0.88687669]
Bootstrapping
Lower bounds: [0.66413767 0.76167205] 
Upper bounds: [0.93133911 1.01799339]
Analytical
Lower bounds: [0.6693117 0.7553747] 
Upper bounds: [0.94425444 1.01837867]


In [6]:
# Check that Torch IG calculation gives same result as analytic
Rtorch = torch.ones_like(torch.Tensor(X))*X.mean(0).reshape(1,-1)
Rset = torch.utils.data.TensorDataset(Rtorch)
EGM = ExpectedGradientsModel(TL.model,Rset,k=10,random_alpha=False,scale_by_inputs=True)
preds,igs = EGM(torch.Tensor(X),shap_values=True)
print("Torch IG:", igs.abs().mean(0).detach().numpy())

Torch IG: [0.8067829 0.8868767]


### See if Torch GLRT matches analytic/bootstrap
We get GLRT CIs for the coefs, then the attributions. We then manually check ***the upper bound of each answer for the (value/attribution) of coefficient 0*** by finding the model parameters that gave this value, checking that the MSE is within the GLRT bounds, and then checking that the parameters do include the coefficient or IG value reported.

In [7]:
# GLRT method with Torch model (coefs)
TL = TorchLinear(lr=0.3,max_iter=30,fit_intercept=False)
TL.fit(X,y)
print("Analytic")
lcb_TL_a, ucb_TL_a = analyticLinearCis(LR, X, y, alpha=0.05)
print("Lower bounds:", lcb_TL_a, "\nUpper bounds:", ucb_TL_a)

print("GLRT")
lcb_GLRT, ucb_GLRT, lcb_Results, ucb_Results, lcb_Torch, ucb_Torch = glrtTorchCis(
    lambda:TorchLinear(lr=0.3,max_iter=100,fit_intercept=False), X=X, y=y, citype='coefs', alpha=0.05,search_kwargs={'lmbds':np.logspace(-10,10,101)},fit_kwargs={'lr':0.3,'max_iter':30})
print("Lower bounds:", lcb_GLRT, "\nUpper bounds:", ucb_GLRT)

Analytic
Lower bounds: [0.74339329 0.91984252] 
Upper bounds: [1.0487684  1.24010963]
GLRT
Lower bounds: [0.53877687 0.68697059] 
Upper bounds: [1.25338912 1.47298121]


In [8]:
# Manually check Torch GLRT (coefs) answer
lcb_MSE, ucb_MSE = lcb_Torch, ucb_Torch#bootstrapGLRTcis(lambda:TorchLinear(lr=0.3,max_iter=100), X, y, lambda x,y: np.mean((x-y)**2), alpha=0.05)
(mses, attributions, coefs, biases) = ucb_Results[0]
valid_inds = mses<=ucb_MSE
valid_attribs, valid_coefs, valid_biases = attributions[valid_inds], coefs[valid_inds], biases[valid_inds]
max_ind = np.argmax(valid_attribs)
max_coef, max_bias = valid_coefs[max_ind], valid_biases[max_ind]
print("Torch attribution search found the following model\nmaximizing coefficient value for feature 0:")
print(f'Coefs: {max_coef}, Bias: {max_bias}')
print(f'Corresponding Analytic IG: {integratedGradients_Linear(max_coef,X)}')
print(f'Model MSE {MSE(y,X@max_coef+max_bias):.3f} < MSE UCB {ucb_MSE:.3f}')

Torch attribution search found the following model
maximizing coefficient value for feature 0:
Coefs: [1.2533891  0.88666147], Bias: 0.0
Corresponding Analytic IG: [1.12848177 0.72812811]
Model MSE 1.161 < MSE UCB 1.246


In [9]:
# GLRT method with Torch model (attribs)
TL = TorchLinear(lr=0.3,max_iter=30)
TL.fit(X,y)
print("Analytic")
lcb_TL_a, ucb_TL_a = integratedGradients_LinearAnalytical(LR, X, y, alpha=0.05)
print("Lower bounds:", lcb_TL_a, "\nUpper bounds:", ucb_TL_a)
print("GLRT")
lcb_GLRT, ucb_GLRT, lcb_Results, ucb_Results, lcb_Torch, ucb_Torch = glrtTorchCis(lambda:TorchLinear(lr=0.3,max_iter=100), X=X, y=y, alpha=0.05,search_kwargs={'lmbds':np.logspace(-10,10,101)},fit_kwargs={'lr':0.3,'max_iter':30})
print("Lower bounds:", lcb_GLRT, "\nUpper bounds:", ucb_GLRT)

Analytic
Lower bounds: [0.71083121 0.67245359] 
Upper bounds: [0.99350252 0.95242894]
GLRT
Lower bounds: [0.35878703 0.32831344] 
Upper bounds: [1.34556675 1.29657757]


In [10]:
# Manually check Torch GLRT (attribs) answer
lcb_MSE, ucb_MSE = lcb_Torch, ucb_Torch#bootstrapGLRTcis(lambda:TorchLinear(lr=0.3,max_iter=100), X, y, lambda x,y: np.mean((x-y)**2), alpha=0.05)
(mses, attributions, coefs, biases) = ucb_Results[0]
valid_inds = mses<=ucb_MSE
valid_attribs, valid_coefs, valid_biases = attributions[valid_inds], coefs[valid_inds], biases[valid_inds]
max_ind = np.argmax(valid_attribs)
max_coef, max_bias = valid_coefs[max_ind], valid_biases[max_ind]
print("Torch attribution search found the following model\nmaximizing attribution value for feature 0:")
print(f'Coefs: {max_coef}, Bias: {max_bias}')
print(f'Corresponding Analytic IG: {integratedGradients_Linear(max_coef,X)}')
print(f'Model MSE {MSE(y,X@max_coef+max_bias):.3f} < MSE UCB {ucb_MSE:.3f}')

Torch attribution search found the following model
maximizing attribution value for feature 0:
Coefs: [1.4582624  0.67052597], Bias: 0.004671680741012096
Corresponding Analytic IG: [1.34556671 0.53622409]
Model MSE 1.277 < MSE UCB 1.282


## Other examples

In [9]:
# Do an example where the coefficient CI's cross zero:
X, y = linearRegression_normal(beta=np.array([0, 1]).T,
                               cov=np.array([[1, 0.5],[0.5, 1]]),
                               sigma=1,
                               n=200)

In [10]:
LR = LinearRegression()
LR.fit(X, y)
print("Coefficients:", LR.coef_)
lcb_LR_coef, ucb_LR_coef = bootstrapCis(LinearRegression, X, y, alpha=0.05, replicates=1000)
print("Coefficient CI's (analytical)", "\nLower bounds:", lcb_LR_coef, "\nUpper bounds:", ucb_LR_coef)

print("Integrated gradients:", integratedGradients_Linear(LR.coef_, X))

print("Bootstrapping")
lcb_LR, ucb_LR = bootstrapIntegratedGradients_Linear(LinearRegression, X, y, alpha=0.05, replicates=1000)
print("Lower bounds:", lcb_LR, "\nUpper bounds:", ucb_LR)

print("Analytical")
lcb_LR_a, ucb_LR_a = integratedGradients_LinearAnalytical(LR, X, y, alpha=0.05)
print("Lower bounds:", lcb_LR_a, "\nUpper bounds:", ucb_LR_a)

Coefficients: [0.03031536 0.97920903]
Coefficient CI's (analytical) 
Lower bounds: [-0.16828497  0.79755857] 
Upper bounds: [0.21636452 1.15397532]
Integrated gradients: [0.02422638 0.81392509]
Bootstrapping
Lower bounds: [0.00322201 0.6459373 ] 
Upper bounds: [0.1917331  0.96169281]
Analytical
Lower bounds: [0.         0.66902844] 
Upper bounds: [0.16020267 0.95882174]


In [11]:
# And an example where the coefficient CI's are negative:
X, y = linearRegression_normal(beta=np.array([-1, 1]).T,
                               cov=np.array([[1, 0.5],[0.5, 1]]),
                               sigma=1,
                               n=200)

In [12]:
LR = LinearRegression()
LR.fit(X, y)
print("Coefficients:", LR.coef_)
lcb_LR_coef, ucb_LR_coef = bootstrapCis(LinearRegression, X, y, alpha=0.05, replicates=1000)
print("Coefficient CI's (analytical)", "\nLower bounds:", lcb_LR_coef, "\nUpper bounds:", ucb_LR_coef)

print("Integrated gradients:", integratedGradients_Linear(LR.coef_, X))

print("Bootstrapping")
lcb_LR, ucb_LR = bootstrapIntegratedGradients_Linear(LinearRegression, X, y, alpha=0.05, replicates=1000)
print("Lower bounds:", lcb_LR, "\nUpper bounds:", ucb_LR)

print("Analytical")
lcb_LR_a, ucb_LR_a = integratedGradients_LinearAnalytical(LR, X, y, alpha=0.05)
print("Lower bounds:", lcb_LR_a, "\nUpper bounds:", ucb_LR_a)

Coefficients: [-1.00387075  1.0247226 ]
Coefficient CI's (analytical) 
Lower bounds: [-1.16141947  0.86636686] 
Upper bounds: [-0.84127545  1.20309793]
Integrated gradients: [0.77268915 0.9136594 ]
Bootstrapping
Lower bounds: [0.64189068 0.76713352] 
Upper bounds: [0.90386435 1.0611927 ]
Analytical
Lower bounds: [0.63028801 0.77010731] 
Upper bounds: [0.9150903  1.05721149]
